In [99]:
import pandas as pd
from thunderpack import ThunderReader
from openpyxl import load_workbook
from openpyxl import load_workbook

In [106]:
patientsDF = pd.read_csv('/media/cdac-c-15/External Drive/Dropbox/cardiac arrest project/cardiac_filtered_bi_with_birthdeath.csv')
patients = set(patientsDF['BDSPPatientID'])
print(len(patientsDF))

1107


In [108]:
patientsDF = patientsDF.drop_duplicates()
print(len(patientsDF))
print(len(patients))

1032
236


In [109]:
#find all transfer records for patients in my group
reader = ThunderReader('/media/cdac-c-15/External Drive/Dropbox/zz_EHR_Thunderpacks/BIDMC/thunderpack_adt_nax_1m_BIDMC')

#find notes that have had an eeg
events = []
for i in range(1, (len(list(reader.keys())) + 1)):
    event = reader[f'adt_partition_{i}']
    event = event[event['bdsppatientid'].isin(patients)]
    events.append(event)
    del event

allEvents = pd.concat(events, axis=0, ignore_index=True)     #puts data into rows and columns
del events

In [110]:
# Filter admit to transfer rows
admit_to_transfer = allEvents[(allEvents['ineventdsc'] == 'admit') & (allEvents['outeventdsc'] == 'transfer')]

# Filter transfer to discharge & admit to transfer
transfer_to_discharge = allEvents[(allEvents['ineventdsc'] == 'transfer') & (allEvents['outeventdsc'] == 'discharge')]
admit_to_discharge = allEvents[(allEvents['ineventdsc'] == 'admit') & (allEvents['outeventdsc'] == 'discharge')]

# Merge the admit_to_transfer and transfer_to_discharge on 'bdspencounterid'
intermediate_merge = pd.merge(admit_to_transfer[['bdspencounterid', 'transferindts', 'bdsppatientid']],
                              transfer_to_discharge[['bdspencounterid', 'transferoutdts', 'bdsppatientid']],
                              on=('bdspencounterid', 'bdsppatientid'),
                              how='inner')

# Combine the direct admit_to_discharge rows with the merged intermediate data
allData = pd.concat( [admit_to_discharge[['bdsppatientid', 'transferindts', 'transferoutdts','bdspencounterid']],intermediate_merge[['bdspencounterid', 'bdsppatientid', 'transferindts', 'transferoutdts']]] , ignore_index=True)
print(allData['bdsppatientid'].nunique())

236


In [111]:
# filter so only patients of interest are included in the resulting dataframe
patientsDF['EEGDate'] = pd.to_datetime(patientsDF['EEGDate'])
allData['transferindts'] = pd.to_datetime(allData['transferindts'])
allData['transferoutdts'] = pd.to_datetime(allData['transferoutdts'])

# Merge the two DataFrames on 'BDSPPatientID'
merged_df = pd.merge(patientsDF, allData, left_on='BDSPPatientID', right_on='bdsppatientid', how='left')

# Filter based on whether 'EEGDate' is between 'transferindts' and 'transferoutdts'
filtered_df = merged_df[(merged_df['EEGDate'] >= merged_df['transferindts']) & (merged_df['EEGDate'] <= merged_df['transferoutdts'])]

# Group by 'BDSPPatientID' and 'EEGDate' to retain only the first match
filtered_df = filtered_df.groupby(['BDSPPatientID', 'EEGDate']).first().reset_index()

# Merge back with the original DataFrame to retain all rows and add the filtered columns
final_df = pd.merge(patientsDF, filtered_df[['BDSPPatientID', 'EEGDate', 'transferindts', 'transferoutdts']], on=['BDSPPatientID', 'EEGDate'], how='left')

# Display the final DataFrame
print(len(final_df))

1032


In [112]:
# Create excel workbook to give rajib with all site 1 and site 2 patients on different pages
path = '/home/cdac-c-15/Desktop/newSpreadsheetWithAdmitDischargeDates.xlsx'

# Create an Excel writer object and write each DataFrame to a different sheet
with pd.ExcelWriter(path, engine='openpyxl') as writer:
    final_df.to_excel(writer, sheet_name='All Patients', index=False)